In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [ ]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from typing import Literal
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time
from math import sqrt

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles() -> dict:

        return {
            "CybP_01": False,  # (Компрометация кода АП) - хэш автопилота
            "CybP_02": False,  # (неполадки связи = смешное сообщение)
            "CybP_03": False,  # привода, поддел инфы о них) - если взлом, то 'eeeeeeee'
            "CybP_04": False,  # (компроментация навиг. системы) - если взлом, то выдает неверны данные
            "CybP_05": False,  # компроментация шлагбаума
            "CybP_06": False,  # постоянно (несанкц. ускорение)
        }
    
    def change_barrier_UDP(self, do: Literal["up"] | Literal["drop"]) -> None:  # Смена положения шлагбаума путем перехвата трафика [UDP] #

        def periodic_ap_barrier():
            future_result = 1 if do == "up" else 0
            while True:
                self.set_barrier_toggle()
                if UserTrustedHandler.check_CybP_05_UDP(self, future_result):
                    break

        ap_barrier = threading.Thread(target=periodic_ap_barrier, daemon=True)
        ap_barrier.start()

    def intercept_UDP(self, name: str) -> int:  # Перехват трафика умных устройств [UDP] #

        try:
            ports = {
            "traffic_0": 5030,  # светофор 0 (самый ближний)
            "traffic_1": 5031,  # светофр 1 (средний)
            "traffic_2": 5032,  # светофор 2 (самый дальний)
            "barrier": 15000,   # шлагбаум
            "autobot": 10004    # автоматичский робот
            }

        except Exception as e:
            self.lg.log(f"Переданный в параметр name аргумент: {name}!")

        listen_port = ports[name]
        result = None

        def periodic_interceptor():
            nonlocal result
            
            interceptor_socket = None
            forward_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.settimeout(0.3)
                interceptor_socket.bind(('127.0.0.1', listen_port))
                
                forward_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                
                try:
                    data, client_address = interceptor_socket.recvfrom(1024)
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name in tuple(ports.keys())[:-2]:  # светофоры
                        color = int(decoded_data[-2])
                        self.lg.log(f"{color}")
                        self.lg.log(f"{name} КРАСНЫЙ") if color == 1 else self.lg.log(f"{name} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name in tuple(ports.keys())[-2]:  # шлагбаум
                        status = decoded_data[-2]
                        result = int(status)
                    elif name in tuple(ports.keys())[-1]:  # автоматичский робот
                        result = decoded_data

                except socket.timeout:
                    self.lg.log("Отключение от порта, ДАННЫЕ НЕ ПОЛУЧЕНЫ!!!")
                    
                except Exception as e:
                    self.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                self.lg.log(f"Ошибка создания сокета: {e}")      
          
            finally:
                try:
                    if interceptor_socket:
                        interceptor_socket.close()
                    if forward_socket:
                        forward_socket.close()
                except:
                    pass
        
        interceptor_thread = threading.Thread(target=periodic_interceptor, daemon=True)
        interceptor_thread.start()
        interceptor_thread.join(timeout=0.25)
        return result

    def check_autobot_UDP(self) -> bool:  # Проверка положения автобота перехватом трафика [UDP] #
        
        def check_distance() -> bool:  # првоеряет факт увеличения расстояния между роботами
            flag = False
            while True:
                for m in self.get_message_from_trusted_module():
                    if m["type"] == "my_coords":
                        my_x, my_y = m["coords"]["position_x"], m["coords"]["position_y"]
                        flag = True
                        break
                if flag:
                    break
            
            check = []
            for _ in range(2):        
                data_autorobot = self.intercept_UDP("autobot")[1:-1].split(',')
                coords = []
                for i in range(2):
                    coord = float(data_autorobot[i].split(':')[1])
                    coords.append(coord)
                    if i == 1:
                        time.sleep(1)
                autorobor_x, autorobor_y = coords[0], coords[1]
                d = round(sqrt((autorobor_x - my_x)**2 + (autorobor_y - my_y)**2), 4)
                check.append(d)

            # self.lg.log(f"{check}")
            return check[0] < check[1]  # возвращает True, если расстоние между роботами уменьшается


        flag_stop = False
        while True:  # если робот стоит на месте; если двигается -> запускаем другой цикл и ждем остановки
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                if not flag_stop:
                    flag_stop = True
                    self.lg.log("[📡<OP>🛜]::💤 Пока что авторобот стоит на месте. Ожидаем... 🛑")
                continue
            else:
                self.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении. Ожидаем... 🛑")
                break
        
        check = False   
        while True:
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                self.lg.log("[📡<OP>🛜]::💤 Авторобот остановился, можно ехать ♻️")
                break
            elif not check:
                result_d = [check_distance() for _ in range(2)]
                if all(result_d):
                    self.lg.log("[📡<OP>🛜]::🌀 Авторобот находится в движении - он уезжает от нас! Вперед... 🟢")
                    break
                check = True

        return True

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 20
        green = False
        red = False
        self.lg.log("🚦 Внимание! Светофор 🚦")

        while True:
            frame = self.get_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(40-velocity, 40+velocity)) and (g in range(200-velocity, 200+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(40-velocity, 40+velocity)) and (g in range(40-velocity, 40+velocity)) and (r in range(200-velocity, 200+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                if not green:
                    green = True
                    self.lg.log("🟢 Светофор ЗЕЛЁНЫЙ. Вперед! 🟢")
                break
            else:
                if not red:
                    red = True
                    self.lg.log("🔴 Светофор КРАСНЫЙ. Ожидаем... 🔴")
                time.sleep(1.0)

        return True 

    def check_pedestrian_CV(self) -> bool:  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (65, 110, 200)  # приблизительные bgr для конуса без тени
        BGR_TARGET_2 = (80, 80, 80)  # приблизительные bgr для конуса в тени
        x, y = 323, 426
        velocity = 55
        wait = False
        
        while True:
            frame = self.get_camera_frame()
            pixels = frame[y, x-2:x+3]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels]
            mas_2 = [[pix[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)] for pix in pixels]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            
            if result_1 or result_2:
                self.lg.log(f"🚶‍♂️Пешеход перешел дорогу. Вперед!🚶‍♂️")
                break
            else:
                if not wait:
                    wait = True
                    self.lg.log(f"🚸 Пешеходный переход. Ожидаем... 🚸")
                time.sleep(0.2)
                continue
        return True

    def change_barrier_CV(self) -> int:  # Смена положения шлагбаума [КЗ] #
        
        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30

        def check_status():
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    return 0
            return 1

        Flag = True
        while Flag:
            buffer = self.get_message_from_trusted_module()
            if len(buffer):
                for message in buffer:
                    if "type" in message.keys():
                        if message["type"] == "status_barrier":
                            start_status =  message["status"]
                            Flag = False
                            break
                if Flag:
                    # self.lg.log("Нет необходимых сообщений!")
                    time.sleep(0.2)
            else:
                # self.lg.log("Буффер пустой!")
                time.sleep(0.2)
        
        flag = False
        while True:
            self.set_barrier_toggle()
            time.sleep(1.7)  # ждем ответ от МБ (процесс поднятия + время получения ответа)
            while not flag:
                buffer = self.get_message_from_trusted_module()
                self.lg.log(f'{buffer}')
                for m in buffer:
                    if m["type"] == "status_barrier":
                        flag = True
                        current_status = m["status"]
                        break
            if start_status != current_status or start_status != check_status():
                break
   

                    

        # message_thread = threading.Thread(target=_buffer_message_handler, daemon=True)
        # message_thread.start()



        def robot_marshrut():
            self.lg.log("Сообщение обычное")
            self.lg.warn("Сообщение о предупреждении")
            self.lg.error("Сообщение об ошибке")

            self.lg.log("Тестовое сообщение блока кода для выполнения заезда")
            self.set_robot_speed(0.24)
            self.set_brush_speed(100)
            # self.change_barrier_UDP("drop")  # Сразу закрываем шлагбаум дистанционно с помощью пакетов
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.68, "y": 2.35})  # Пешеход верхний
            self.check_pedestrian_CV()
            self.change_barrier_UDP("drop")  # Снова закрываем шлагбаум дистанционно с помощью пакетов

            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # Возможное столкновение с роботом (первый чек)
            self.check_autobot_UDP()
            
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.7, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.6, "y":3.4})
            self.ap_hook.do_move({"x": 2.16, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
            self.ap_hook.do_move({"x": 1.6, "y":3.08}) #
            self.ap_hook.do_rotate({"x": 1.32, "y":2.12}) #
            self.ap_hook.do_move({"x": 1.32, "y":2.12}) #

            self.ap_hook.do_rotate({"x": 1.32, "y":2.12})
            self.ap_hook.do_move({"x": 1.32, "y":2.12})
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 2.6, "y":2.12})
            self.ap_hook.do_rotate({"x": 2.65, "y":2.2})
            self.ap_hook.do_move({"x": 2.65, "y":2.2})
            self.ap_hook.do_rotate({"x": 2.6, "y":2.28})
            self.ap_hook.do_move({"x": 2.6, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.2, "y":2.28})
            self.ap_hook.do_move({"x": 1.8, "y":2.28})  # Светофор 1 (центральный)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.36, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.32, "y":0.8})
            self.ap_hook.do_move({"x": 1.32, "y":1})  # Светофор 0 (нижний)
            self.check_t_light_CV()

            self.ap_hook.do_move({"x": 1.32, "y":0.8})
            self.ap_hook.do_rotate({"x": 1.4, "y":0.6})
            self.ap_hook.do_move({"x": 1.4, "y":0.6})
            self.ap_hook.do_rotate({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1.4})
            self.ap_hook.do_move({"x": 1.48, "y":1.8})
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.6})  # Возможное столкновение с роботом (второй чек)
            self.check_autobot_UDP()
 
            self.ap_hook.do_move({"x": 1.48, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.ap_hook.do_move({"x": 0.52, "y":2.85})  # Верхний пешехов        
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.52, "y":1.8})
            self.ap_hook.do_move({"x": 0.52, "y":1.4})
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.ap_hook.do_rotate({"x": 0.52, "y":0.52})
            self.ap_hook.do_move({"x": 0.52, "y":0.52})
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1, "y":0.52})

            self.ap_hook.do_move({"x": 1.4, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52}) 
            self.ap_hook.do_move({"x": 1.9, "y":0.52})
            self.ap_hook.do_move({"x": 1.95, "y":0.52})  # Пешеход нижний 
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.ap_hook.do_move({"x": 3.08, "y":0.52})
            self.ap_hook.do_rotate({"x": 3.08, "y":1})  # Возможное столкновение с роботом (третий чек)
            self.check_autobot_UDP()

            self.ap_hook.do_move({"x": 3.08, "y":1})
            self.ap_hook.do_move({"x": 3.08, "y":1.4})
            self.ap_hook.do_move({"x": 3.08, "y":1.8})
            self.ap_hook.do_move({"x": 3.08, "y":2.2})
            self.ap_hook.do_move({"x": 3.08, "y":2.6})
            self.ap_hook.do_move({"x": 3.08, "y":3.0})
            # self.check_autobot_UDP() !!! НЕ ЗДЕСЬ !!!
            self.ap_hook.do_rotate({"x": 3, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.92, "y":3})
            self.ap_hook.do_move({"x": 2.92, "y":3})
            self.ap_hook.do_rotate({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":0.68})
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
            self.ap_hook.do_move({"x": 0.6, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0}) 

        robot_marshrut()

    def check_CybP_05_CV(self) -> bool:  # Активность CybP_05 [КЗ] #
        
        """ Запускается при попадании в ячейки шлагбаума (29, 47)
            примерно через 1.5 секунды после попадания в них """

        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30
        open = False
        close = False
        
        def check_status():
            nonlocal open, close
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    if not close:
                        close = True
                        self.lg.log(f"⛔️ Проверка положения шлагбаума...Шлагбаум ЗАКРЫТ ⛔️")
                    return 0
            if not open:
                open = True
                self.lg.log(f"🍀 Проверка положения шлагбаума...ОТКРЫТ 🍀")
            return 1
        
        time.sleep(1.5)
        start_status = check_status()
        self.send_message_to_ap({"type": "status_barrier", "status": start_status})
        flag = False

        while True:
            time.sleep(1.5)  # запрос на смену положения, отправляемый из АП (процесс поднятия шлагбаума)
            current_status = check_status()
            self.send_message_to_ap({"type": "status_barrier", "status": current_status})
            time.sleep(0.3)
            self.send_message_to_ap({"type": "status_barrier", "status": current_status})

            if start_status != current_status:
                if not flag:
                    self.lg.log("📡 Положение шлагбаума ИЗМЕНЕНО. Спасибо, Диспетчер 📡")
                    break
                else:
                    self.lg.log("📡 Положение шлагбаума ИЗМЕНЕНО. Спасибо, Диспетчер 📡")
                    self.lg.log("✅ CybP_05: Защита применена")
                    break
            else:
                if not flag:
                    flag = True
                    self.lg.log("🚨 ОБНАРУЖЕНА КОМПРОМЕНТАЦИЯ ШЛАГБАУМА CybP_05! 🚨")
                else:
                    self.lg.log("   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌")

    def check_CybP_05_UDP(ap: any, future_result: int) -> bool:  # Активность CybP_05 [UDP] #
            time.sleep(0.3)
            if future_result == ap.intercept_UDP("barrier"):
                ap.lg.log("[📡<OP>🛜]::🍀 Положение шлагбаума ИЗМЕНЕНО 🍀")
                if UserTrustedHandler.get_last_instance().cybp_05_udp:
                    ap.lg.log("[📡<OP>🛜]::✅ CybP_05: Защита на уровне Диспетчера применена")
                UserTrustedHandler.get_last_instance().cybp_05_udp = False
                return True
            else:
                if not UserTrustedHandler.get_last_instance().cybp_05_udp:
                    UserTrustedHandler.get_last_instance().cybp_05_udp = True
                    ap.lg.log("[📡<OP>🛜]::🚨 ОБНАРУЖЕНА КОМПРОМЕНТАЦИЯ ШЛАГБАУМА CybP_05! 🚨")
                else:
                    ap.lg.log("[📡<OP>🛜]::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌")
                return False

class UserTrustedHandler(TrustedHandler):
    
    def __init__(self, context):
        super().__init__(context)

    def trusted_code(self):
        
        self._check_robot_speed_and_position()

    def make_next_short_message(self, prev_message: str):
        
        self.cybp02_message_count += 1
        self.cybp02_protection_active = True
        
        self._detect_cybp02_attack(prev_message)
        
        return self.last_correct_message

### 
                
    def _check_robot_speed_and_position(self):
            
        def monitor():
            
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status["position_x"]
                current_y = robot_status["position_y"]
                rotation = robot_status["rotation"]
                

                self._calculate_cell_from_position(current_x, current_y, int(rotation))
                    # current_time = time.time()
                    # self._calculate_robot_speed(current_x, current_y, current_time)
                    

                    # if self.cybp06_current_speed > self.cybp06_speed_threshold:
                    #     self._handle_cybp06_speed_attack()

        while True:
            monitor_start = threading.Thread(target=monitor, daemon=True)
            monitor_start.start()

    def _calculate_cell_from_position(self, x, y, rotation):
                
        cell_x = x // 0.4 + 1
        cell_y = 9 - (y // 0.4 + 1)
        
        cell_number = int(cell_y * 9 + cell_x)

        # МТС
        if cell_number in (12, 22, 71): 
            self.send_message_to_ap({"type": "my_coords", "coords": self.get_robot_status()})

        # ШЛАГБАУМ
        elif (cell_number == 47 and rotation in range(80, 91)) \
            or (cell_number == 29 and rotation in range(-90, -87)) \
                and not self.check_barrier: 
            if not self.check_barrier:
                self.check_barrier = True 
                self.check_CybP_05_CV()
        
        if cell_number not in (29, 47):  
            self.check_barrier = False

        return cell_number


In [3]:
start_module(UserMissionHandler, UserTrustedHandler)

[01:54]::INIT::AGTS-WRAP-CLIENT, версия 2.0.0
[01:54]::INIT::UDP(c) <- (uid=tBHC) инициализирован.
[01:54]::INIT::UDP(c) <- (uid=mBRP) инициализирован.
[01:54]::INIT::Загрузка завершена.
[01:54]::INIT::UDP(c) -> (uid=nEnA) инициализирован.
[01:54]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[01:54]::INFO::(AP) Заезд инициализирован - ожидание старта
[01:54]::INFO::(TM) Заезд инициализирован - ожидание старта
[01:54]::INFO::Код заезда инициализирован
[01:54]::INFO::Сообщение обычное
[01:54]::WARN::Сообщение о предупреждении
[01:54]::ERROR::Сообщение об ошибке
[01:54]::INFO::Тестовое сообщение блока кода для выполнения заезда
[01:54]::INFO::Доверенный код инициализирован
[01:55]::INFO::[]
[01:55]::INFO::Буффер пустой!
[01:55]::INFO::[]
[01:55]::INFO::Буффер пустой!
[01:55]::INFO::⛔️ Проверка положения шлагбаума...Шлагбаум ЗАКРЫТ ⛔️
[01:55]::INFO::[{'type': 'status_barrier', 'status': 0}]
[01:55]::INFO::🚨 ОБНАРУЖЕНА КОМПРОМЕНТАЦИЯ ШЛАГБАУМА CybP_05! 🚨
[01:55]::INFO::[{

[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРНОЕ СРАБАТЫВАНИЕ КОМПРОМЕНТАЦИИ ШЛАГБАУМА CybP_05! 🎲❌
[01:55]::INFO::   ❌🎲 ПОВТОРН